In [1]:
import collections
from etils import epath
from ml_collections import config_dict
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from chirp.inference import a2o_utils
from chirp.inference import interface
from chirp.inference import tf_examples
from chirp.inference.search import bootstrap
from chirp.inference.search import search
from chirp.inference.search import display
from chirp.inference.classify import classify

2024-05-10 07:55:50.516898: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-10 07:55:50.517037: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-10 07:55:50.791990: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-10 07:55:57.782591: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model_choice = 'perch'

working_dir = '.'

labeled_data_path = epath.Path(working_dir) / 'labeled'
custom_classifier_path = epath.Path(working_dir) / 'bird-vocalization-classifier'

embeddings_path = 'embeddings'

bootstrap_config = bootstrap.BootstrapConfig.load_from_embedding_path(
    embeddings_path=embeddings_path,
    annotated_path=labeled_data_path)

cfg = config_dict.ConfigDict({
    'model_path': custom_classifier_path,
    'logits_key': 'custom',
})

loaded_model = interface.LogitsOutputHead.from_config(cfg)
model = loaded_model.logits_model
class_list = loaded_model.class_list

In [3]:
species_list = pd.read_csv('./custom_classifier/birds.csv')['species_code'].tolist()

In [4]:
output_filepath = epath.Path(working_dir) / 'big_classifier_outputs/inference_all.csv'

class_thresholds = None

exclude_classes = []
include_classes = species_list

embeddings_ds = tf_examples.create_embeddings_dataset(
    embeddings_path)

In [5]:
embeddings_ds.element_spec

{'embedding': TensorSpec(shape=<unknown>, dtype=tf.float32, name=None),
 'embedding_shape': TensorSpec(shape=(None,), dtype=tf.int64, name=None),
 'filename': TensorSpec(shape=(), dtype=tf.string, name=None),
 'frontend': TensorSpec(shape=<unknown>, dtype=tf.float32, name=None),
 'frontend_shape': TensorSpec(shape=(None,), dtype=tf.int64, name=None),
 'raw_audio': TensorSpec(shape=<unknown>, dtype=tf.float32, name=None),
 'raw_audio_shape': TensorSpec(shape=(None,), dtype=tf.int64, name=None),
 'separated_audio': TensorSpec(shape=<unknown>, dtype=tf.float32, name=None),
 'separated_audio_shape': TensorSpec(shape=(None,), dtype=tf.int64, name=None),
 'timestamp_s': TensorSpec(shape=(), dtype=tf.float32, name=None)}

In [6]:
classify.write_inference_csv(
    embeddings_ds=embeddings_ds,
    model=model,
    labels=class_list.classes,
    output_filepath=output_filepath,
    threshold=class_thresholds,
    embedding_hop_size_s=bootstrap_config.embedding_hop_size_s,
    include_classes=include_classes,
    exclude_classes=exclude_classes)

TypeError: in user code:

    File "/home/mschulist/miniconda3/lib/python3.10/site-packages/chirp/inference/classify/classify.py", line 162, in classify_batch  *
        logits = model(flat_emb)

    TypeError: '_UserObject' object is not callable
